## Homework 3
### Submitted by: Abu Huraira

In [14]:
(require minikanren)
(require minikanren/matche)
(require minikanren/numbers)

1. Binary trees

Consider the following representation of binary trees:
• '() — an empty binary tree;
• `(node ,value ,left ,right) — a node with a value value and two subtrees (left and right).

Exercise 1.1 (⋆, 5 points). Implement a relational program treeo that checks whether a given term is a valid tree.
```racket
    (run* (q) (treeo 'empty))
    ; '(_.0)
    (run* (q) (treeo '(node 1 empty (node 2 (node 3 empty empty) empty))))
    ; '(_.0)
    (run 5 (q) (treeo q))
    ; '(empty
    ; (node _.0 empty empty)
    ; (node _.0 empty (node _.1 empty empty))
    ; (node _.0 (node _.1 empty empty) empty)
    ; (node _.0 empty (node _.1 empty (node _.2 empty empty))))
```

In [15]:
(defrel (treeo tree)
  (conde
    [(== 'empty tree)]
    [(fresh (node val l r)
       (== `(node ,val ,l ,r) tree)
       (treeo l)
       (treeo r))]))

(displayln "Testing treeo...")
(displayln (run* (q) (treeo 'empty)))
(displayln (run* (q) (treeo '(node 1 empty (node 2 (node 3 empty empty) empty)))))
(displayln (run 5 (q) (treeo q)))

Testing treeo...
(_.0)
(_.0)
(empty (node _.0 empty empty) (node _.0 empty (node _.1 empty empty)) (node _.0 (node _.1 empty empty) empty) (node _.0 empty (node _.1 empty (node _.2 empty empty))))


Exercise 1.2 (⋆⋆, 5 points). Implement predicate prefix-subtreeo such that (`prefix-subtreeo small large`) is `#t` when small is contained in large:
1. an empty tree is contained in any tree;
2. a non-empty tree is contained in another non-empty tree when they have the same root and both subtrees are contained in the other subtrees respectively;

```racket
    (run* (q) (prefix-subtreeo q '(node 3 (node 1 empty (node 2 empty empty)) (node 4 empty empty))))
    ; '(empty
    ; (node 3 empty empty)
    ; (node 3 empty (node 4 empty empty))
    ; (node 3 (node 1 empty empty) empty)
    ; (node 3 (node 1 empty empty) (node 4 empty empty))
    ; (node 3 (node 1 empty (node 2 empty empty)) empty)
    ; (node 3 (node 1 empty (node 2 empty empty)) (node 4 empty empty)))
```


In [17]:
(defrel (prefix-subtreeo small large)
  (conde
    [(== 'empty small)]
    [(fresh (n1 v1 l1 r1 n2 v2 l2 r2)
      (== `(node ,v1 ,l1 ,r1) small)
      (== `(node ,v2 ,l2 ,r2) large)
      (== v1 v2)
      (prefix-subtreeo l1 l2)
      (prefix-subtreeo r1 r2))]))


(displayln "Testing prefix-subtreeo...")
(displayln (run* (q) (prefix-subtreeo q '(node 3 (node 1 empty (node 2 empty empty)) (node 4 empty empty)))))

Testing prefix-subtreeo...
(empty (node 3 empty empty) (node 3 empty (node 4 empty empty)) (node 3 (node 1 empty empty) empty) (node 3 (node 1 empty empty) (node 4 empty empty)) (node 3 (node 1 empty (node 2 empty empty)) empty) (node 3 (node 1 empty (node 2 empty empty)) (node 4 empty empty)))


Exercise 1.3 (⋆⋆, 10 points). Implement predicate preordero such that (`preorder tree vals`) is `#t` when vals contains exactly of values from tree in **preorder traversal** order.
```racket
    (run* (q) (preordero '(node 1 (node 2 (node 3 empty empty) empty) (node 4 empty empty)) '(3 2 1 4)))
    ; '()
    (run* (q) (preordero '(node 1 (node 2 (node 3 empty empty) empty) (node 4 empty empty)) q))
    ; '((1 2 3 4))
    (run* (q) (preordero q '(1 2 3)))
    ; '((node 1 empty (node 2 empty (node 3 empty empty)))
    ; (node 1 empty (node 2 (node 3 empty empty) empty))
    ; (node 1 (node 2 empty empty) (node 3 empty empty))
    ; (node 1 (node 2 empty (node 3 empty empty)) empty)
    ; (node 1 (node 2 (node 3 empty empty) empty) empty))
    (run 7 (q) (fresh (xs) (preordero q xs)))
    ; '(empty
    ; (node _.0 empty empty)
    ; (node _.0 empty (node _.1 empty empty))
    ; (node _.0 (node _.1 empty empty) empty)
    ; (node _.0 empty (node _.1 empty (node _.2 empty empty)))
    ; (node _.0 (node _.1 empty empty) (node _.2 empty empty))
    ; (node _.0 empty (node _.1 (node _.2 empty empty) empty)))
```

2 Simple expressions
Exercise 2.1 (⋆, 5 points). Implement predicate expro that checks if a given term is a valid arithmetic expression:
1. a number;
2. a term (+ X Y) where both X and Y are valid expressions;
3. a term (* X Y) where both X and Y are valid expressions;

```racket
    (run* (e) (expro '(+ 2 (* 3 4))))
    ; '(_.0)
    (run* (e) (expro '(+ 2 (* e 4))))
    ; '()
```


In [8]:
(defrel (expro expr)
  (conde
    [(numbero expr)]
    [(fresh (op X Y)
       (== `(+ ,X ,Y) expr)
       (expro X)
       (expro Y))]
    [(fresh (op X Y)
       (== `(* ,X ,Y) expr)
       (expro X)
       (expro Y))]))

(displayln "Testing expro...")
(displayln (run* (e) (expro '(+ 2 (* 3 4)))))
; '(_.0)
(displayln (run* (e) (expro '(+ 2 (* e 4)))))
; '()

Testing expro...
(_.0)
()


Exercise 2.2 (⋆⋆, 10 points). Implement predicate expro-with such that (`expro-with e subexprs`) is `#t` when `e` is an expression term that uses each subexpression from subexprs once (in that order). You may assume that subexprs has known shape (i.e. length):

```racket
    (run* (e) (expro-with e '(1 2)))
    ; '((+ 1 2) (* 1 2))
    (run* (e) (expro-with e '(1 2 3)))
    ; '((+ 1 (+ 2 3))
    ; (+ (+ 1 2) 3)
    ; (+ 1 (* 2 3))
    ; (+ (* 1 2) 3)
    ; (* 1 (+ 2 3))
    ; (* (+ 1 2) 3)
    ; (* 1 (* 2 3))
    ; (* (* 1 2) 3))
```

Exercise 2.3 (⋆, 5 points). Implement a function `equations-with` such that (`equations-with subexprs`) returns a list of equations in the form `(= val expr)` where `expr` is an expression generated from subexpressions `subexpr` and `val` is its corresponding numeric value.
```racket
    (equations-with '(1 2 3))
    ; '((= 6 (+ 1 (+ 2 3)))
    ; (= 6 (+ (+ 1 2) 3))
    ; (= 7 (+ 1 (* 2 3)))
    ; (= 5 (+ (* 1 2) 3))
    ; (= 5 (* 1 (+ 2 3)))
    ; (= 9 (* (+ 1 2) 3))
    ; (= 6 (* 1 (* 2 3)))
    ; (= 6 (* (* 1 2) 3)))
```

Exercise 2.4 (⋆, 5 points). Implement a relational program `atomic-expro` that checks that a given expression is atomic (not a binary operation expression).

```racket
    (run* (q) (fresh (x) (atomic-expro 2)))
    ; '(_.0)
    (run* (q) (fresh (x) (atomic-expro x)))
    ; '(_.0)
    (run* (q) (fresh (x) (atomic-expro `(+ 1 ,x))))
    ; '()
```

In [18]:
(require minikanren)
(require minikanren/matche)
(require minikanren/numbers)

(defrel (atomic-expro t) (numbero t))


(displayln "Testing atomic-expro...")
(displayln (run* (q) (fresh (x) (atomic-expro 2))))
; '(_.0)
(displayln (run* (q) (fresh (x) (atomic-expro x))))
; '(_.0)
(displayln (run* (q) (fresh (x) (atomic-expro `(+ 1 ,x)))))
; '()

Testing atomic-expro...
(_.0)
(_.0)
()


Exercise 2.5 (⋆, 5 points). Implement a relational program `root-equiv-expro` that checks if two expressions are related by one of the following equations directly:
1. x + y = y + x for all x, y
2. x + 0 = x for all x
3. x × 1 = x for all x
4. x × 0 = 0 for all x
5. (x × y) × z = x × (y × z) for all x, y, z
6. x × (y × z) = (x × y) × z for all x, y, z
7. (x + y) + z = x + (y + z) for all x, y, z
8. x + (y + z) = (x + y) + z for all x, y, z
9. (x + y) × z = (x × z) + (y × z) for all x, y, z
10. x × (y + z) = (x × y) + (x × z) for all x, y, z

```racket
    (run* (e) (root-equiv-expro '(+ x (* 0 y)) e))
    ; '((+ (* 0 y) x))
    (run* (e) (root-equiv-expro '(* x (* 0 y)) e))
    ; '((* (* 0 y) x) (* (* x 0) y))
    (run* (e) (root-equiv-expro '(* 1 (* 0 y)) e))
    ; '((* 0 y) (* (* 0 y) 1) (* (* 1 0) y))
```


In [13]:
(defrel (root-equiv-expro e1 e2)
  (conde
    [(fresh (x y)
       (== `(+ ,x ,y) e1)
       (== `(+ ,y ,x) e2))]

    [(fresh (x)
      (== `(+ ,x 0) e1)
     (== x e2))]

    [(fresh (x)
      (== `(* ,x 1) e1)
     (== x e2))]

    [(fresh (x)
      (== `(* ,x 0) e1)
     (== 0 e2))]

    [(fresh (x y z)
       (== `(* (* ,x ,y) ,z) e1)
       (== `(* ,x (* ,y ,z)) e2))]

    [(fresh (x y z)
       (== `(* ,x (* ,y ,z)) e1)
       (== `(* (* ,x ,y) ,z) e2))]

    [(fresh (x y z)
       (== `(+ (+ ,x ,y) ,z) e1)
       (== `(+ ,x (+ ,y ,z)) e2))]

    [(fresh (x y z)
       (== `(+ ,x (+ ,y ,z)) e1)
       (== `(+ (+ ,x ,y) ,z) e2))]

    [(fresh (x y z)
       (== `(* (+ ,x ,y) ,z) e1)
       (== `(+ (* ,x ,z) (* ,y ,z)) e2))]

    [(fresh (x y z)
       (== `(* ,x (+ ,y ,z)) e1)
       (== `(+ (* ,x ,y) (* ,x ,z)) e2))]))


(displayln (run* (e) (root-equiv-expro '(+ x (* 0 y)) e)))
; '((+ (* 0 y) x))
(displayln (run* (e) (root-equiv-expro '(* x (* 0 y)) e)))
; '((* (* 0 y) x) (* (* x 0) y))
(displayln (run* (e) (root-equiv-expro '(* 1 (* 0 y)) e)))
; '((* 0 y) (* (* 0 y) 1) (* (* 1 0) y))

((+ (* 0 y) x))
((* (* x 0) y))
((* (* 1 0) y))


Exercise 2.6 (⋆, 5 points). Implement a relational program `equiv-expro` that checks if two expressions are related by one of the equations above, possibly somewhere in a subterm:

```racket
    (run* (e) (equiv-expro '(+ x (* 0 y)) e))
    ; '((+ (* 0 y) x) (+ x 0) (+ x (* y 0)))
    (run* (e) (equiv-expro '(* x (* 0 y)) e))
    ; '((* (* 0 y) x) (* (* x 0) y) (* x 0) (* x (* y 0)))
    (run* (e) (equiv-expro '(* 1 (* 0 y)) e))
    ; '((* 0 y) (* (* 0 y) 1) (* (* 1 0) y) (* 1 0) (* 1 (* y 0)))
```